# ABS Monthly Household Spending Indicator 5682

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data
from mgplot import seastrend_plot_finalise, series_growth_plot_finalise, line_plot_finalise, bar_plot_finalise, finalise_plot

# display charts in this notebook
SHOW = False

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("5682.0")
plot_times = 0, -61
print(meta[mc.table].unique())

['5682001' '5682002' '5682003' '5682004' '5682005' '5682006' '5682007'
 '5682008' '5682009' '5682010']


## Plotting

### Headline charts

In [3]:
def headline_charts() -> None:
    """Produce headline charts."""

    wanted_tables = ["5682001"]

    for table in wanted_tables:
        if table not in abs_dict:
            print(f"Skipping {table} as it is not in the data")
            continue

        dids = meta.loc[
            (meta[mc.table] == table)
            & (meta[mc.did].str.contains("Household spending"))
            # & (~meta[mc.did].str.contains("ercent"))
            & (meta[mc.stype] == "Seasonally Adjusted"),
            mc.did,
        ].unique()
        for did in dids:
            select = {
                table: mc.table,
                did: mc.did,
                "Seasonally Adjusted": mc.stype,
            }
            _, seas_adj_id, units = ra.find_abs_id(meta, select, verbose=False)
            del select["Seasonally Adjusted"]
            select["Trend"] = mc.stype
            _, trend_id, units = ra.find_abs_id(meta, select, verbose=False)
            frame = pd.DataFrame()
            frame["Seasonally Adjusted"] = abs_dict[table][seas_adj_id]
            frame["Trend"] = abs_dict[table][trend_id]
            if "ercent" not in did:
                frame, units = ra.recalibrate(frame, units)
            elif "previous period" in did:
                # the COVID shock drowns out everything else
                frame = frame.loc[frame.index >= "2023"]
            title = "\n".join(did.split(";", 1))
            seastrend_plot_finalise(
                frame,
                title=(title.replace(" ; ", ",").replace(" ;", "")),
                ylabel=units,
                y0=True,
                lfooter="Australia. Monthly data. ",
                rfooter="ABS 5682.0",
                show=SHOW,
                rounding=1,
                dropna=False,  # Trend is missing data
            )

            if "percentage" in title.lower():
                continue

            series_growth_plot_finalise(
                frame["Seasonally Adjusted"],
                plot_from=-19,
                title=title,
                y0=True,
                lfooter="Australia. Monthly data. ",
                rfooter="ABS 5682.0",
                tag="Growth",
                show=SHOW,
            )


headline_charts()

### Comparison with HFCE in National Accounts

In [4]:
def get_hfce() -> tuple[pd.Series, str]:

    cat = "5206.0"
    table = "5206008_Household_Final_Consumption_Expenditure"
    stype = "Seasonally Adjusted"
    d, m = ra.read_abs_cat(cat=cat, single_excel_only=table)
    selector = {
        table: mc.table,
        stype: mc.stype,
        "FINAL CONSUMPTION EXPENDITURE: Current prices ;": mc.did,
    }
    _table, sid, units = ra.find_abs_id(m, selector, verbose=False)
    series = d[table][sid]
    return series, units

In [5]:
def m_to_q(m: pd.Series, qfreq="Q-DEC") -> pd.Series:
    """Convert monthly data to quarterly data by summing.
    Assume index is a Monthly PeriodIndex.
    Only return data in respect of complete quarters.
    Returns a Series with a Quarterly PeriodIndex."""

    q = m.to_timestamp().resample("QE").sum()
    count = m.to_timestamp().resample("QE").count()
    q = q[count == 3]  # only keep complete quarters
    q.index = q.index.to_period(qfreq)

    return q

In [6]:
def spending() -> tuple[pd.Series, str]:
    "Get the ABS monthly household spending indicator data."

    did = (
        "Household spending ;  Total (Household Spending Categories) ;"
        + "  Australia ;  Current Price ;"
    )
    stype = "Seasonally Adjusted"
    table = "5682001"
    selector = {
        table: mc.table,
        did: mc.did,
        stype: mc.stype,
    }
    _table, sid, units = ra.find_abs_id(meta, selector, verbose=False)
    m_indicator = abs_dict[table][sid]

    return m_indicator, units

In [7]:
def comparison() -> None:
    """Compare household spending and consumption."""

    hfce, hfce_units = get_hfce()
    m_spend, spend_units = spending()
    q_spend = m_to_q(m_spend)
    assert hfce_units == spend_units
    data = pd.DataFrame(
        {
            "Household Final Consumption Expenditure": hfce,  # 0
            "Household Spending Indicator": q_spend,  # 1
        }
    ).dropna(how="any")
    data, units = ra.recalibrate(data, hfce_units)

    common_plot = {
        "lfooter": "Australia. Seasonally Adjusted. Current Prices.",
        "rfooter": "ABS 5682.0 5602.0",
        "show": SHOW,
    }

    line_plot_finalise(
        data,
        title="Household Spending and Consumption",
        ylabel=f"{units} / Quarter",
        **common_plot,
    )

    bar_data = data[data.columns[1]] / data[data.columns[0]] * 100.0
    common_plot.update(
        {
            "title": "Household Spending as a Percentage of Consumption",
            "ylabel": "Percentage",
        }
    )
    bar_plot_finalise(
        bar_data,
        bar_legend=False,
        tag="bar",
        annotate=True,
        rotation=90,
        **common_plot,
    )


comparison()

plot_then_finalise: Keyword argument validation issues:
bar_legend: keyword not recognised. Has argument=False in
plot_then_finalise.



### Compare Household spending with Retail Turnover

In [8]:
def retail_turnover() -> tuple[pd.Series, str]:
    "Get the ABS monthly retail turnover data."

    cat = "8501.0"
    table = "850101"
    d, m = ra.read_abs_cat(cat=cat, single_excel_only=table)
    did = "Turnover ;  Total (State) ;  Total (Industry) ;"
    selector = {
        table: mc.table,
        did: mc.did,
        "Seasonally Adjusted": mc.stype,
    }
    _table, sid, units = ra.find_abs_id(m, selector, verbose=False)
    series = d[table][sid]
    return series, units

In [9]:
def comparison2() -> None:
    """Retail Turnover in compariosn with Household Spending."""

    m_spend, spend_units = spending()
    m_turnover, turnover_units = retail_turnover()
    assert turnover_units == spend_units
    data = pd.DataFrame(
        {
            "Monthly Retail Turnover": m_turnover,  # 0
            "Household Spending Indicator": m_spend,  # 1
        }
    ).dropna(how="any")
    data, units = ra.recalibrate(data, turnover_units)

    common_plot = {
        "lfooter": "Australia. Seasonally Adjusted. Current Prices.",
        "rfooter": "ABS 5682.0 8501.0",
        "show": SHOW,
    }

    line_plot_finalise(
        data,
        title="Household Spending and Retail Turnover",
        ylabel=f"{units} / Month",
        width=2,
        annotate=True,
        **common_plot,
    )
    bar_data = data[data.columns[0]] / data[data.columns[1]] * 100.0
    common_plot["title"] = "Retail Turnover as a Percentage of Household Spending"
    common_plot["ylabel"] = "Percentage"
    bar_plot_finalise(
        bar_data,
        tag="bar",
        legend=False,
        **common_plot,
        annotate=True,
    )
    line_plot_finalise(
        bar_data,
        tag="line",
        width=2,
        **common_plot,
    )


comparison2()

## Finished

In [10]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-06-16 17:42:39

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 9.3.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

readabs: 0.0.31
pandas : 2.3.0
mgplot : 0.1.13

Watermark: 2.5.0



In [11]:
print("Done")

Done
